In [74]:
import struct
import numpy as np

In [75]:
def load_idx_images(path):
    with open(path, 'rb') as f:
        magic, num, rows, cols = struct.unpack(">IIII", f.read(16))
        assert magic == 2051, f"Magic number mismatch, got {magic}"
        data = np.frombuffer(f.read(), dtype=np.uint8)
        # flat the data to N * 784 matrix and change to float32
        images = (data.reshape(num, rows*cols)).astype(np.float32) / 255.0
        return images

In [76]:
def load_idx_labels(path):
    with open(path, "rb") as f:
        magic, num = struct.unpack(">II", f.read(8))
        assert magic == 2049, f"Magic number mismatch, got {magic}"
        data = np.frombuffer(f.read(), dtype=np.uint8)
        return data

In [77]:
def one_hot(labels: np.ndarray):
    targets = np.zeros((labels.size, 10), dtype=np.float32)
    #生成一个长度为labels长度的索引数列
    index_matrix= np.arange(labels.size)
    #用labels向量的个数来做行索引，用它的值来做列索引
    targets[index_matrix, labels] = 1
    return targets

In [78]:
# 前向传播函数，得到Z
def forward(X: np.ndarray, W: np.ndarray, b: np.ndarray):
    Z = X.dot(W) + b
    return Z

In [79]:
# softmax处理，把Z变成概率矩阵
def softMax(Z: np.ndarray):
    Z_shift = Z - np.max(Z, axis=1, keepdims=True)
    exp_Z = np.exp(Z_shift)
    probs = exp_Z / np.sum(exp_Z, axis=1, keepdims=True)
    return probs

In [80]:
# 计算损失函数的Y_onehot版本
def cross_entropy_from_onehot(Y_hat: np.ndarray, Y_onehot: np.ndarray, eps= 1e-12):
    logp = np.log(Y_hat + eps) #给矩阵每个元素加一个微小的eps，避免无穷，然后把每个数变成log
    # 逐元素相乘：把 logp 和 Y_onehot 对位相乘。
    loss = -np.sum(Y_onehot * logp) / Y_hat.shape[0] #把矩阵所有值相加，但是不为零的只有对应位置，所以是所有正确概率log相加除以个数
    # 得到的就是损失函数
    return loss

In [81]:
# 计算损失函数的y_int版本
def cross_entropy_from_int(Y_hat: np.ndarray, y_int: np.ndarray, eps: int=1e-12):
    # 生成一个y_hat长度的index索引
    rows = np.arange(Y_hat.shape[0])
    # 使用高级索引，得到y_hat中所有需要得到的值，也就是对应正确答案的概率
    p_true = Y_hat[rows, y_int]
    # 把所有正确答案加eps求log，然后求负平均值
    loss = -np.mean(np.log(p_true + eps))
    return loss

In [82]:
# 计算损失函数对Zt的导数，作为求得梯度的前提
def d_loss_d_Z(Y_hat: np.ndarray, Y_onehot: np.ndarray, B: int):
    G = Y_hat.copy()
    G -= Y_onehot
    G /= B
    return G

In [ ]:
# 读取文件
images = load_idx_images("./train-images.idx3-ubyte")
labels = load_idx_labels("./train-labels.idx1-ubyte")

# 设定训练样本大小
batch_size = 128
# 设定训练轮次
epochs = 20
# 设定学习率
N = images.shape[0]
lr = 0.1

In [84]:
# 创建参数矩阵
rows, cols = 784, 10
W = (np.random.randn(rows, cols) * 0.01).astype(np.float32)
b = np.zeros((1, 10), dtype=np.float32)

In [85]:
# #求的概率矩阵
# Y_hat= softMax(Z)
# assert np.allclose(Y_hat.sum(axis=1), 1.0, atol=1e-6)
# G = d_loss_d_Z(Y_hat, Y_onehot, B)
# # 最重要的，得到对于W和对于b的偏导数，从而得到梯度
# grad_W = Xb.T.dot(G)
# grad_b = G.sum(axis=0, keepdims=True)
# W -= lr * grad_W
# b -= lr * grad_b

In [86]:
for epoch in range(epochs):
    indices = np.random.permutation(N)
    X_shuffled = images[indices]
    Y_shuffled = labels[indices]
    
    for start in range(0, N, batch_size):
        end = start + batch_size
        Xb = X_shuffled[start:end]
        Yb = Y_shuffled[start:end]
        B = Xb.shape[0] 

        # forward 
        Z = Xb @ W + b
        probs = softMax(Z)

        # reverse
        Y_onehot = one_hot(Yb)
        G = (probs - Y_onehot) / B
        grad_W = Xb.T @ G
        grad_b = G.sum(axis=0, keepdims=True)

        # update W and b
        W -= lr * grad_W
        b -= lr * grad_b

    Z_all = images @ W + b
    probs_all = softMax(Z_all)
    loss = cross_entropy_from_int(probs_all, labels)
    acc = (probs_all.argmax(axis = 1) == labels).mean()
    print(f"round {epoch + 1}: loss={loss:.4f}, acc={acc:.4f}")

round 1: loss=0.4021, acc=0.8917
round 2: loss=0.3517, acc=0.9032
round 3: loss=0.3305, acc=0.9073
round 4: loss=0.3188, acc=0.9105
round 5: loss=0.3087, acc=0.9141
round 6: loss=0.3019, acc=0.9164
round 7: loss=0.2979, acc=0.9168
round 8: loss=0.2929, acc=0.9188
round 9: loss=0.2905, acc=0.9192
round 10: loss=0.2869, acc=0.9209
round 11: loss=0.2847, acc=0.9210
round 12: loss=0.2823, acc=0.9217
round 13: loss=0.2796, acc=0.9226
round 14: loss=0.2792, acc=0.9229
round 15: loss=0.2765, acc=0.9239
round 16: loss=0.2757, acc=0.9242
round 17: loss=0.2734, acc=0.9245
round 18: loss=0.2718, acc=0.9247
round 19: loss=0.2708, acc=0.9251
round 20: loss=0.2704, acc=0.9249
round 21: loss=0.2689, acc=0.9255
round 22: loss=0.2679, acc=0.9255
round 23: loss=0.2671, acc=0.9262
round 24: loss=0.2662, acc=0.9262
round 25: loss=0.2657, acc=0.9264
round 26: loss=0.2652, acc=0.9261
round 27: loss=0.2642, acc=0.9272
round 28: loss=0.2637, acc=0.9274
round 29: loss=0.2629, acc=0.9277
round 30: loss=0.2618, 

In [87]:
X_test_path = "./t10k-images.idx3-ubyte"
Y_test_path = "./t10k-labels.idx1-ubyte"
X_test = load_idx_images(X_test_path)
Y_test = load_idx_labels(Y_test_path)

Z_test = X_test @ W + b
probs_test = softMax(Z_test)
loss_test = cross_entropy_from_int(probs_test, Y_test)
acc_test = (probs_test.argmax(axis=1) == Y_test).mean()
print(f"\ntest result: loss: {loss_test:4f}, acc: {acc_test:4f}.")
np.save("W.npy", W)
np.save("b.npy", b)


test result: loss: 0.267029, acc: 0.924100.
